# Random Forests


In [1]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, r2_score, mean_squared_error
import pandas as pd
from binning import bin

Read the train and test datasets


In [2]:
train_df = pd.read_csv("Training_set.csv")
test_df = pd.read_csv("Testing_set.csv")

Get the training features and label


In [3]:
X_train = train_df.iloc[:, :-1].to_numpy()
y_train = train_df.iloc[:, -1].to_numpy()

Get the testing features and label


In [4]:
X_test = test_df.iloc[:, :-1].to_numpy()
y_test = test_df.iloc[:, -1].to_numpy()

## Random Forests Classifier


Bin the label


In [6]:
y_train_binned = bin(y_train)
y_test_binned = bin(y_test)

Initialize a RandomForestClassifier


In [7]:
rfc = RandomForestClassifier(verbose=3, n_jobs=4)

Train the model


In [ ]:
rfc.fit(X_train, y_train_binned)

Make the predictions on the test set


In [ ]:
y_pred = rfc.predict(X_test)

Evaluate the predictions


In [10]:
print(classification_report(y_true=y_test_binned, y_pred=y_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.35      0.02      0.04     11718
           1       0.49      0.78      0.60     42216
           2       0.44      0.37      0.40     32447
           3       0.60      0.09      0.15      9998

    accuracy                           0.48     96379
   macro avg       0.47      0.32      0.30     96379
weighted avg       0.47      0.48      0.42     96379



## Random Forests Regressor


Initialize a RandomForestRegressor


In [5]:
rfr = RandomForestRegressor(verbose=3, n_jobs=4)

Train the model


In [ ]:
rfr.fit(X_train, y_train)

Make the predictions on the test set


In [ ]:
y_pred = rfr.predict(X_test)

Evaluate the predictions


In [8]:
print(f"R2 Score = {r2_score(y_true=y_test, y_pred=y_pred)}")
print(f"MSE = {mean_squared_error(y_true=y_test, y_pred=y_pred)}")

R2 Score = 0.1015225935847417
MSE = 2951.2007601375817
